In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import glob
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from nilearn.image import resample_to_img
from factor_analyzer import FactorAnalyzer

Initialize

In [ ]:
export_figures = True
PATH = "..."
scaler = StandardScaler()
imp = SimpleImputer(missing_values=999,strategy="median")

In [ ]:
# Define the colors
color = ['#3f005c', '#ffa600']
my_colors = []
for c in color:
    my_colors.append(matplotlib.colors.to_rgb(c))

Prepare data

In [ ]:
# Load lesion load data
lesion_load = pd.read_csv(PATH + "data.csv")
lesion_load = lesion_load.drop(columns="Unnamed: 0")
lesion_bilat = lesion_load.iloc[:1401].copy()
lesion_bilat = lesion_bilat.rename(columns={"0": "Left"})
lesion_bilat["Right"] = np.array(lesion_load.iloc[1401:])

In [ ]:
# Load behavioral data
Behav = pd.read_excel(PATH + 'behavioral.xlsx')
Behav = Behav[np.invert((lesion_bilat!=0).all(axis=1))]  # excluding patients with bilateral lesions
# Imputing missing values with median
Behav_imp = pd.DataFrame(imp.fit_transform(Behav))
Behav_imp.columns = Behav.columns
# Average time for TMT A and B
Behav_imp['TMT_Time_zscore_neg'] = np.mean([Behav_imp.TMT_A_Time_zscore_neg, Behav_imp.TMT_B_Time_zscore_neg], axis=0)

In [ ]:
# Extra behavioral measures
Behav_extra = pd.read_excel(PATH + 'behav_extended.xlsx')
Behav_extra = Behav_extra[Behav_extra.ID.isin(Behav_imp.ID)]
Behav_extra_imp = pd.DataFrame(imp.fit_transform(Behav_extra))
Behav_extra_imp.columns = Behav_extra.columns
Behav_imp['SemanticFluency'] = scaler.fit_transform(Behav_extra_imp.Semanticfluencyanimal.values.reshape(-1, 1))
Behav_imp['PhonemicFluency'] = scaler.fit_transform(Behav_extra_imp.Phonemicfluency_total.values.reshape(-1, 1))
Behav_imp['DigitSymbolCoding'] = scaler.fit_transform(Behav_extra_imp.DigitSymbolCoding_correct.values.reshape(-1, 1))
Behav_imp = Behav_imp.drop(columns=["ID","Cohort"])
# Scaling the total infarct volume
Total_infarct_volume_scaled = scaler.fit_transform(np.array(Behav_imp.Total_infarct_volume).reshape(-1, 1))
Behav_imp["Total_infarct_volume_scaled"] = Total_infarct_volume_scaled
# Collapsing the data
Behav_imp_collapsed = pd.concat([Behav_imp, Behav_imp], axis=0)
Behav_imp_collapsed.index = range(0,2160)

In [ ]:
# Load lesion masks
subject_names = glob.glob(PATH + '/lesionmaps/*',)
subject_names = sorted(subject_names)
# Load atlas
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr50-2mm', symmetric_split=True)
cortical_mask = dataset.maps
cortical_labels = dataset.labels
dataset = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr50-2mm')
subcortical_mask = dataset.maps
subcortical_labels = dataset.labels
# Edit the labels
cortical_labels = pd.DataFrame(cortical_labels)
cortical_labels = cortical_labels.drop(index=[0,93,94])
subcortical_labels = pd.DataFrame(subcortical_labels)
subcortical_labels = subcortical_labels.drop(index=[0,1,2,3,12,13,14])

In [ ]:
# Loading mapped to cortical and subcortical regions
cortical = pd.read_csv(PATH + "/cortical.csv")
cortical = cortical.drop(columns="Unnamed: 0")
subcortical = pd.read_csv(PATH + "/subcortical.csv")
subcortical = subcortical.drop(columns="Unnamed: 0")

# resample one sub to the cortical mask
cortical_resampled_stat_img = resample_to_img(cortical_mask, subject_names[0], interpolation='nearest')
# resample one subject to subcortical mask
subcortical_resampled_stat_img = resample_to_img(subcortical_mask, subject_names[0], interpolation='nearest')
# count voxels in a region
cortical_count_labels = np.bincount(cortical_resampled_stat_img.get_data().astype(np.int64).ravel())
subcortical_count_labels = np.bincount(subcortical_resampled_stat_img.get_data().astype(np.int64).ravel())

In [ ]:
# Extracting the labels
cortical_count_labels = pd.DataFrame(cortical_count_labels)
cortical_count_labels = np.array(cortical_count_labels.drop(index=[0,93,94])).ravel()
subcortical_count_labels = pd.DataFrame(subcortical_count_labels)
subcortical_count_labels = np.array(subcortical_count_labels.drop(index=[0, ])).ravel()

# number of lesion in each region
cortical_lesion_count_absolute = cortical*cortical_count_labels
subcortical_lesion_count_absolute = subcortical*subcortical_count_labels
subcortical_lesion_count_absolute = subcortical_lesion_count_absolute.drop(columns=["0","1","2","11","12","13"])

In [ ]:
# Renaming the columns
cortical_lesion_count_absolute.columns = cortical_labels
subcortical_lesion_count_absolute.columns = subcortical_labels

new_columns = []
for parcel in range(len(cortical_lesion_count_absolute.columns)):
        new_columns.append(cortical_lesion_count_absolute.columns[parcel][0].replace(" ", "_"))
cortical_lesion_count_absolute.columns = new_columns

new_columns = []
for parcel in range(0, len(subcortical_lesion_count_absolute.columns)):
        new_columns.append(subcortical_lesion_count_absolute.columns[parcel][0].replace(" ", "_"))
subcortical_lesion_count_absolute.columns = new_columns

subcortical_lesion_count_absolute = subcortical_lesion_count_absolute.drop(columns=['Brain-Stem'])

# reduce to non-bilateral
cortical_lesion_count_absolute = cortical_lesion_count_absolute[np.invert((lesion_bilat!=0).all(axis=1))]

In [ ]:
# Splitting the data into left and right hemisphere
cortical_lesion_left = cortical_lesion_count_absolute.iloc[:,range(0,94,2)]
cortical_lesion_left.loc[:,"Left_hemisphere"] = 0
cortical_lesion_right = cortical_lesion_count_absolute.iloc[:,range(1,95,2)]
cortical_lesion_right.loc[:,"Left_hemisphere"] = 1

for hem, hem_name in zip([cortical_lesion_right, cortical_lesion_left], ["Right_", "Left_"]):
    new_columns = []
    for parcel in range(0, len(hem.columns[:-1])):
            new_columns.append(hem.columns[parcel].replace(hem_name, ""))
    new_columns.append("Left_hemisphere")
    hem.columns = new_columns

cortical_lesion_collapsed = pd.concat([cortical_lesion_left, cortical_lesion_right])

In [ ]:
# Splitting the data into left and right hemisphere
subcortical_lesion_count_absolute = subcortical_lesion_count_absolute[np.invert((lesion_bilat!=0).all(axis=1))]

subcortical_lesion_left = subcortical_lesion_count_absolute.iloc[:,range(0,7,1)]
subcortical_lesion_left.loc[:,"Left_hemisphere"] = 0
subcortical_lesion_right = subcortical_lesion_count_absolute.iloc[:,range(7,14,1)]
subcortical_lesion_right.loc[:,"Left_hemisphere"] = 1

for hem, hem_name in zip([subcortical_lesion_right, subcortical_lesion_left], ["Right_", "Left_"]):
    new_columns = []
    for parcel in range(0, len(hem.columns[:-1])):
            new_columns.append(hem.columns[parcel].replace(hem_name, ""))
    new_columns.append("Left_hemisphere")
    hem.columns = new_columns

subcortical_lesion_collapsed = pd.concat([subcortical_lesion_left, subcortical_lesion_right])

In [ ]:
lesions_collapsed_all = pd.concat([cortical_lesion_collapsed, subcortical_lesion_collapsed], axis=1)
lesions_collapsed_hemisphere = lesions_collapsed_all["Left_hemisphere"].copy()
lesions_collapsed_hemisphere = np.array(lesions_collapsed_hemisphere)
lesions_collapsed_all = lesions_collapsed_all.drop(columns="Left_hemisphere")


In [ ]:
hemisphere_idx = lesions_collapsed_hemisphere[:,0]
n_hemispheres = len(np.unique(lesions_collapsed_hemisphere))
n_components = 10

Prepare covariates

In [ ]:
# Age mean scaled
Age_mean_scaled = Behav_imp["Age"] - Behav_imp["Age"].mean()
Age_mean_scaled = np.concatenate((Age_mean_scaled, Age_mean_scaled))
Age_mean_scaled_2 = np.array(Age_mean_scaled * Age_mean_scaled)

Behav_imp_collapsed["Sex_1_m"] = Behav_imp_collapsed["Sex"].replace(1,0)
Behav_imp_collapsed["Sex_1_m"] = Behav_imp_collapsed["Sex_1_m"].replace(2,1)
Sex_m = np.int64(np.array(Behav_imp_collapsed["Sex_1_m"]))
Behav_imp_collapsed["Sex_1_w"] = Behav_imp_collapsed["Sex"].replace(2,0)
Sex_w = np.int64(np.array(Behav_imp_collapsed["Sex_1_w"]))

Age_Sex_m = Age_mean_scaled * Sex_m
Age_Sex_w = Age_mean_scaled * Sex_w

Lesion_vol_scaled = np.array(Behav_imp_collapsed.Total_infarct_volume_scaled)

In [ ]:
Education_mean_scaled = Behav_imp["Education_years"] - Behav_imp["Education_years"].mean() 
Education_mean_scaled = np.concatenate((Education_mean_scaled, Education_mean_scaled))
Education_mean_scaled = np.array(Education_mean_scaled)

IQ_mean_scaled = Behav_imp["IQCODE"] - Behav_imp["IQCODE"].mean()
IQ_mean_scaled = np.concatenate((IQ_mean_scaled , IQ_mean_scaled ))
IQ_mean_scaled = np.array(IQ_mean_scaled )

In [ ]:
# Outlier detection and removal
X1 = Behav_imp_collapsed.MMSE_total_zscore.values
X2 = Behav_imp_collapsed.ReyComplexFigureTestCopy_zscore.values
X3 = Behav_imp_collapsed.BostonNamingTest_zscore.values
X4 = Behav_imp_collapsed.Seoul_Verbal_Learning_Test_immediate_recall_total_zscore.values
X5 = Behav_imp_collapsed.TMT_Time_zscore_neg.values
X6 = Behav_imp_collapsed.SemanticFluency.values
X7 = Behav_imp_collapsed.PhonemicFluency.values
X8 = Behav_imp_collapsed.DigitSymbolCoding.values
X_all = np.array([X1, X2, X3, X4, X5, X6, X7, X8])

# Input names
cat_names = ['MiniMentalStateExam', 'ReyComplexFigure', 'BostonNaming',
             'SeoulVerbalLearning', 'TrailMaking', 'SemanticFluency',
             'PhonemicFluency', 'DigitSymbolCoding']

Factor analysis

In [ ]:

n_comp = 4
fa = FactorAnalyzer(n_factors=n_comp, rotation="varimax", method='principal')
X_star = fa.fit_transform(X_all.T)

A_ = fa.loadings_

sns.set_theme(style="white", palette='colorblind', font_scale=1.8)
fig, ax = plt.subplots(1, figsize=(8, 6))
g = sns.heatmap(data=(A_), cbar=True, cmap='RdBu_r',  linewidths=0.5,
                square=False, annot=False, center=0)
                # vmin=0.2, vmax=0.8,)
g.set_yticklabels(cat_names, rotation=0)
g.set_xticklabels(np.arange(1,n_comp+1), rotation=0)
g.set_ylabel('')
g.set_xlabel('Dimension')
ax.set_title('Factor loadings matrix', fontsize=22)
plt.show()

In [ ]:
n_comp = 8
fa = FactorAnalyzer(n_factors=n_comp, rotation="varimax")
X_star = fa.fit_transform(X_all.T)
d = fa.get_factor_variance()
ev = np.array(d[1]*100)
eig = fa.get_eigenvalues()

# Create scree plot using matplotlib
d = pd.DataFrame({'factor': np.arange(0, len(ev)),
                  'ev': np.cumsum(ev),
                  'eig': eig[0]})
sns.set_theme(style="white", palette='colorblind', font_scale=2.2)
fig, ax = plt.subplots(1, figsize=(12, 6))
sns.barplot(x='factor',y='ev', data=d, ax=ax, color=color[0])
plt.ylabel('Cum. expl. variance [%]', fontsize=24)
plt.xlabel('Factors', fontsize=26)
ax2 = ax.twinx()
sns.scatterplot(x='factor',y='eig', data=d, ax=ax2, color=color[1], s=80)
ax2.plot(d.factor, d.eig, c=color[1],linewidth=2.5)
ax2.set_yticks(np.array([1, 2, 3]))
plt.ylabel('Eigenvalues', fontsize=24)
plt.title('', fontsize=22)
plt.xlabel('Factors', fontsize=26)
ax.set_xticklabels(np.arange(1, len(ev)+1))
ax.set_yticks(np.array([20, 40, 60]))
ax.grid()
sns.despine(left=True)
plt.show()